# Spaceship Titanic Competition

### Goals: do some work with the categorical columns, one hot encode

To Do List - check if model can absorb nulls, if not convert floats to int (can't do that until there are no nulls)

In [19]:
import pandas as pd

final_data = pd.read_csv('test.csv')
train_data = pd.read_csv('train.csv')

In [20]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


setting features

In [41]:
features = train_data.columns.to_list()
features

['PassengerId',
 'HomePlanet',
 'CryoSleep',
 'Cabin',
 'Destination',
 'Age',
 'VIP',
 'RoomService',
 'FoodCourt',
 'ShoppingMall',
 'Spa',
 'VRDeck',
 'Name',
 'Transported']

In [28]:
from sklearn.model_selection import train_test_split

X = train_data[features]
y = train_data.Transported

X_train, X_valid, y_train, y_valid = train_test_split(X, y)

list of all categorical columns

In [65]:
cat_cols = X_train.select_dtypes(include='object').columns.to_list()
cat_cols

['PassengerId',
 'HomePlanet',
 'CryoSleep',
 'Cabin',
 'Destination',
 'VIP',
 'Name']

(code from the kaggle tutorial - checking to see how many unique values are in each categorical column)

In [66]:
# Get number of unique entries in each column with categorical data
object_nunique = list(map(lambda col: X_train[col].nunique(), cat_cols))
d = dict(zip(cat_cols, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])

[('CryoSleep', 2),
 ('VIP', 2),
 ('HomePlanet', 3),
 ('Destination', 3),
 ('Cabin', 5148),
 ('Name', 6357),
 ('PassengerId', 6519)]

In [73]:
usecat = ['CryoSleep', 'VIP', 'HomePlanet', 'Destination']

-------------------------------------------------------------------------------------------------------------------------------

setting up for imputation

In [47]:
num = train_data._get_numeric_data()
num.drop(['Transported'], axis=1, inplace=True)
numdata = num.columns.to_list()

slightly modified code from the kaggle tutorial

In [48]:
from sklearn.ensemble import RandomForestClassifier

(I'm just splitting up a cell here so I can see what is taking so long)

https://stackoverflow.com/questions/67361786/how-to-measure-random-forest-classifier-accuracy

In [49]:
from sklearn.impute import SimpleImputer
from sklearn import metrics

In [50]:
i = SimpleImputer()
i_X_train = pd.DataFrame(i.fit_transform(X_train[numdata]))
i_X_valid = pd.DataFrame(i.transform(X_valid[numdata]))

i_X_train.columns = X_train[numdata].columns
i_X_valid.columns = X_valid[numdata].columns

model = RandomForestClassifier(random_state=1, n_estimators=100, criterion='entropy')
model.fit(i_X_train, y_train)
preds = model.predict(i_X_valid)
print("Accuracy:", metrics.accuracy_score(y_valid, preds))

Accuracy: 0.7672493100275989


-------------------------------------------------------------------------------------------------------------------------------

In [52]:
ff = final_data[features]
i_X_final = pd.DataFrame(i.transform(ff))
i_X_final.columns = final_data[numdata].columns

final_preds = model.predict(i_X_final)

KeyError: "['Transported'] not in index"

In [43]:
output = pd.DataFrame({'PassengerId': final_data.PassengerId, 'Transported': final_preds})
output.to_csv('submission1.csv', index=False)